In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import classification_report

In [2]:
# from sklearn.model_selection import train_test_split
from model_selection import train_test_split

In [3]:
df = pd.read_csv('../data/waterQuality1.csv')
# https://www.kaggle.com/datasets/mssmartypants/water-quality/data

In [4]:
df = df.drop(df[df['ammonia'] == "#NUM!"].index)
df['ammonia'] = df['ammonia'].astype('float')
df['is_safe'] = df['is_safe'].astype('float')

In [5]:
# df.info()

In [6]:

x = np.array(df.drop(['is_safe'], axis=1))
y = np.array(df['is_safe'].copy())

In [7]:
x_train,x_test,y_train,y_test = train_test_split(x,y, testsize=0.2, debug=True)
x_train


Data  Size: 7996
Train Size: 6397
Test  Size: 1599
Train Size + Test Size: 7996


array([[4.000e-02, 9.350e+00, 1.000e-02, ..., 9.000e-02, 2.000e-02,
        3.000e-02],
       [2.000e-02, 2.464e+01, 6.000e-02, ..., 4.000e-02, 0.000e+00,
        8.000e-02],
       [7.000e-02, 1.996e+01, 6.700e-01, ..., 3.000e-02, 3.800e-01,
        2.000e-02],
       ...,
       [8.000e-02, 1.504e+01, 0.000e+00, ..., 9.000e-02, 9.000e-02,
        8.000e-02],
       [7.000e-02, 6.360e+00, 4.000e-02, ..., 1.000e-02, 5.000e-02,
        2.000e-02],
       [2.460e+00, 1.514e+01, 7.500e-01, ..., 9.000e-02, 1.300e-01,
        7.000e-02]])

In [8]:
model = svm.SVC(kernel='linear')
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
model.score(x_test, y_test)


print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# print("Precision:",metrics.precision_score(y_test, y_pred, average="weighted"))
# print("Recall:",metrics.f1_score(y_test, y_pred, average="macro"))
print(classification_report(y_test, y_pred))


Accuracy: 0.9111944965603502
              precision    recall  f1-score   support

         0.0       0.92      0.99      0.95      1404
         1.0       0.82      0.35      0.49       195

    accuracy                           0.91      1599
   macro avg       0.87      0.67      0.72      1599
weighted avg       0.90      0.91      0.90      1599

